In [ ]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
country = 'ML'
use_gcs_cache = False
dask_scheduler = "threads"

In [ ]:
from pyprojroot import here
import geopandas as gpd
import dask
dask.config.set(scheduler=dask_scheduler);

The map below shows regions where mosquito genomic data are available and genome-wide 
selection scans have been performed.

Click on a region to see more information about mosquito cohorts for which data are 
available.

In [ ]:
gdf_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson").set_index('cohort_id')
gdf_cohorts = gdf_cohorts.query(f"country_alpha2 == '{country}'")

In [ ]:
from ipyleaflet import Map, Marker, basemaps
from ipywidgets import HTML

center = gdf_cohorts[['latitude', 'longitude']].mean().to_list()
m = Map(center=center, zoom=8, basemap=basemaps.OpenTopoMap)

for coh_id, row in gdf_cohorts.iterrows():
    lat, long = row[['latitude', 'longitude']]
    coh_label = row['cohort_label']
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'
    
    marker = Marker(location=(lat, long), draggable=True, opacity=0.7, color=color)
    m.add_layer(marker);
    
    message2 = HTML()
    message2.value = f'<a href="../cohort/{coh_id}.html">{coh_label}</a>'
    marker.popup = message2

m

<br></br>

In [ ]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)

In [ ]:
gdf_cohorts = gdf_cohorts.reset_index()
gdf_cohorts['url'] = "../cohort/" + gdf_cohorts['cohort_id'] + ".html"
gdf_cohorts['cohort_label'] = gdf_cohorts.apply(lambda x: make_clickable(x['url'], x['cohort_label']), axis=1)

gdf_cohorts = gdf_cohorts[['cohort_label', 'cohort_size', 'admin1_name', 'admin2_name', 'taxon', 'year', 'quarter']]
gdf_cohorts = gdf_cohorts.rename(columns={'cohort_label':'Cohort label'}).set_index('Cohort label')
gdf_cohorts.columns = ['Cohort size', 'Admin1 area', 'Admin2 area', 'Taxon', 'Year', 'Quarter']
gdf_cohorts.style